## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-23-13-11-00 +0000,wsj,Waller Weighs Supporting Fed Rate Pause if Lab...,https://www.wsj.com/articles/waller-weighs-sup...
1,2026-02-23-13-09-12 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/02/23/world/...
2,2026-02-23-13-08-00 +0000,wsj,Gold Gains 2% on Haven Demand,https://www.wsj.com/finance/commodities-future...
3,2026-02-23-13-07-23 +0000,nyt,What to Know About the Killing of ‘El Mencho’,https://www.nytimes.com/2026/02/22/world/ameri...
4,2026-02-23-13-05-41 +0000,wapo,Teen surfers rescue family from cold water aft...,https://www.washingtonpost.com/lifestyle/2026/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2504/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
55,trump,44
56,new,14
428,iran,14
47,court,13
48,tariff,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
140,2026-02-23-02-00-00 +0000,wsj,With the bulk of President Trump’s tariffs str...,https://www.wsj.com/economy/trade/tariffs-are-...,129
121,2026-02-23-03-16-14 +0000,nyt,Armed Man Shot Dead at Mar-a-Lago by Trump’s S...,https://www.nytimes.com/2026/02/22/us/north-ca...,104
233,2026-02-22-14-09-27 +0000,cbc,"Armed man shot, killed after entering perimete...",https://www.cbc.ca/news/world/trump-mar-a-lago...,104
218,2026-02-22-17-17-27 +0000,nypost,C-SPAN caller using Trump’s pseudonym and rema...,https://nypost.com/2026/02/22/us-news/c-span-c...,98
20,2026-02-23-12-08-15 +0000,nypost,Trump invites gold-winning US men’s Olympic ho...,https://nypost.com/2026/02/23/us-news/trump-in...,93


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
140,129,2026-02-23-02-00-00 +0000,wsj,With the bulk of President Trump’s tariffs str...,https://www.wsj.com/economy/trade/tariffs-are-...
90,71,2026-02-23-08-37-41 +0000,nypost,Rocket launchers among military-grade equipmen...,https://nypost.com/2026/02/23/world-news/rocke...
114,70,2026-02-23-04-30-52 +0000,nypost,"Austin Tucker Martin, gunman shot dead by Secr...",https://nypost.com/2026/02/22/us-news/austin-t...
20,49,2026-02-23-12-08-15 +0000,nypost,Trump invites gold-winning US men’s Olympic ho...,https://nypost.com/2026/02/23/us-news/trump-in...
40,37,2026-02-23-11-28-02 +0000,nypost,Ex-Prince Andrew charged taxpayers for massage...,https://nypost.com/2026/02/23/world-news/princ...
132,35,2026-02-23-02-29-00 +0000,wsj,Mexico Takes on Cartels as Killing of Drug Kin...,https://www.wsj.com/world/americas/mexican-sec...
236,33,2026-02-22-14-00-40 +0000,bbc,Iran students stage first large anti-governmen...,https://www.bbc.com/news/articles/c5yj2kzkrj0o...
105,28,2026-02-23-06-42-06 +0000,nypost,Grieving North Carolina dad attacks teen son’s...,https://nypost.com/2026/02/23/us-news/north-ca...
45,27,2026-02-23-11-13-00 +0000,wsj,This Country’s Hot Stock Market Won’t Help You...,https://www.wsj.com/finance/stocks/this-countr...
21,27,2026-02-23-12-04-00 +0000,wsj,President Trump sent Netflix a clear-ish signa...,https://www.wsj.com/politics/policy/trumps-omi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
